# eNanoMapper API guide

## eNanoMapper database background

- FP7 project eNanoMapper http://www.enanomapper.net/
- eNanoMapper database implementation: AMBIT software http://ambit.sf.net
- publication https://www.beilstein-journals.org/bjnano/articles/6/165
- eNanoMapper prototype database https://data.enanomapper.net 

## Nanosafety data 

- NanoSafety data compiled in eNanoMapper databases: https://search.data.enanomapper.net/ 
- Each project data is imported into one eNanoMapper instance, e.g. https://apps.ideaconsult.net/nanoreg1  
  - AMBIT REST API 
- Aggregated search view across multiple databases are available at https://search.data.enanomapper.net 
  - Solr REST API


## eNanoMapper data model

![data model](http://ambit.sourceforge.net/enanomapper/templates/images/data_model.png)
http://ambit.sourceforge.net/enanomapper/templates/convertor_how.html

## eNanoMapper database API

Swagger API docs at http://enanomapper.github.io/API/

In [2]:
from solrscope import aa
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import requests
from importlib import reload 
from solrscope import client_solr
from solrscope import client_ambit
from solrscope import annotation
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

In [3]:
print('Select eNanoMapper database instance:')

#config="enanomapper_private.yaml"
config="enanomapper_public.yaml"
#config="enm_composite_private.yaml"

def search_service_protected(url,apikey):
    return (url,apikey)
def search_service_open(url):
    return (url)    


style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3(config=config)    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)


Select eNanoMapper database instance:


interactive(children=(Dropdown(description='Service:', options=('https://apps.ideaconsult.net/nanoreg1', 'http…

### What is in the database ?

In [4]:
service_uri=service_widget.value
if auth_object != None:
    auth_object.setKey(apikey_widget.value)

cli_facets = client_ambit.AMBITFacets(service_uri)
r = cli_facets.get(page=0,pagesize=1000,auth=auth_object)
if r.status_code==200:

    facets = cli_facets.parse(r.json())    
    print(json.dumps(facets, indent=4))   
else:
    facets = None
    print(r.status_code)

[
    {
        "value": "6.1.5. Toxicity to aquatic algae and cyanobacteria",
        "endpoint": "EC_ALGAETOX_SECTION",
        "count": 20,
        "substancescount": -1,
        "uri": "https://apps.ideaconsult.net/nanoreg1/substance?type=endpointcategory&search=EC_ALGAETOX_SECTION",
        "subcategory": "ECOTOX",
        "subcategoryuri": "https://apps.ideaconsult.net/nanoreg1/substance?type=topcategory&search=ECOTOX",
        "bundles": {}
    },
    {
        "value": "6.1.3. Short-term toxicity to aquatic inverterbrates",
        "endpoint": "EC_DAPHNIATOX_SECTION",
        "count": 30,
        "substancescount": -1,
        "uri": "https://apps.ideaconsult.net/nanoreg1/substance?type=endpointcategory&search=EC_DAPHNIATOX_SECTION",
        "subcategory": "ECOTOX",
        "subcategoryuri": "https://apps.ideaconsult.net/nanoreg1/substance?type=topcategory&search=ECOTOX",
        "bundles": {}
    },
    {
        "value": "6.3.1. Toxicity to soil macroorganisms",
        "endp

In [5]:
df=pd.DataFrame(facets)
display(df[["subcategory","endpoint","value","count"]])

,subcategory,endpoint,value,count
0,ECOTOX,EC_ALGAETOX_SECTION,6.1.5. Toxicity to aquatic algae and cyanobact...,20
1,ECOTOX,EC_DAPHNIATOX_SECTION,6.1.3. Short-term toxicity to aquatic inverter...,30
2,ECOTOX,EC_SOILDWELLINGTOX_SECTION,6.3.1. Toxicity to soil macroorganisms,12
3,P-CHEM,ANALYTICAL_METHODS_SECTION,CHMO_0001075. Analytical Methods,85
4,P-CHEM,ASPECT_RATIO_SHAPE_SECTION,4.27. Nanomaterial aspect ratio/shape,13
5,P-CHEM,CRYSTALLINE_PHASE_SECTION,4.25. Nanomaterial crystalline phase,118
6,P-CHEM,CRYSTALLITE_AND_GRAIN_SIZE_SECTION,4.26. Nanomaterial crystallite and grain size,6
7,P-CHEM,DUSTINESS_SECTION,4.31. Nanomaterial dustiness,22
8,P-CHEM,ENM_0000081_SECTION,ENM_0000081. Batch Dispersion quality,168
9,P-CHEM,ENM_8000223_SECTION,ENM_8000223. Aerosol characterisation,29


In [6]:
#endpoints
cli_facets = client_ambit.AMBITFacets(service_uri,key="/experiment_endpoints")

r = cli_facets.get(page=0,pagesize=100,params={"top":"TOX"},auth=auth_object)
if r.status_code==200:

    facets = cli_facets.parse(r.json())    
    #print(json.dumps(facets, indent=4))   
    df=pd.DataFrame(facets)
    display(df)
else:
    substances = None
    print(r.status_code)

,top,category,value,endpoint,endpointtype,unit,synonyms,count
0,TOX,ENM_0000037_SECTION,ENM_0000037. Oxidative Stress,PERCENTAGE_OF_CONTROL,DOSERESPONSE,%,[http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,191
1,TOX,ENM_0000044_SECTION,ENM_0000044. Barrier integrity,AVERAGE EPITHELIAL HEIGHT PER TISSUE,DOSERESPONSE,um,[],60
2,TOX,ENM_0000044_SECTION,ENM_0000044. Barrier integrity,AVERAGE OF DEEPEST PENETRATION DEPTH (PER TISSUE),DOSERESPONSE,um,[],58
3,TOX,ENM_0000044_SECTION,ENM_0000044. Barrier integrity,INTEGRITY CONTROL QUALITY: MEAN TEER VALUES BE...,AGGREGATED,ohms/cm2,[],65
4,TOX,ENM_0000044_SECTION,ENM_0000044. Barrier integrity,MAXIMUM TEER VALUES BEFORE STARTING NPS EXPOSURE,AGGREGATED,ohms/cm2,[http://purl.enanomapper.org/onto/ENM_8000301],65
...,...,...,...,...,...,...,...,...
95,TOX,TO_GENETIC_IN_VITRO_SECTION,7.6.1. Genetic toxicity in vitro,PERCENTAGE_OF_CONTROL,None,None,[http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,96
96,TOX,TO_GENETIC_IN_VITRO_SECTION,7.6.1. Genetic toxicity in vitro,REFERENCE CONCENTRATION USED FOR NORMALISATION...,AGGREGATED,ug/ml,[],60
97,TOX,TO_GENETIC_IN_VITRO_SECTION,7.6.1. Genetic toxicity in vitro,REPLICATION INDEX,None,"C+,C-,C1,C2,C3,C4",[],96
98,TOX,TO_GENETIC_IN_VITRO_SECTION,7.6.1. Genetic toxicity in vitro,RESULT,None,None,[],892


###  Substance queries
#### All gold nanoparticles

In [7]:
materialtype="NPO_401"

a = annotation.DictionarySubstancetypes()
print(">>> Looking for {}".format(a.annotate(materialtype)))

service_uri=service_widget.value

cli_materials = client_ambit.AMBITSubstance(service_uri)
r = cli_materials.get(params={'search': materialtype,'type' : 'substancetype'},page=0,pagesize=10,auth=auth_object)
if r.status_code==200:

    substances = cli_materials.parse(r.json())    
    print(json.dumps(substances, indent=4))    
else:
    substances = None
    print(r.status_code)

>>> Looking for gold nanoparticle
[
    {
        "URI": "https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602",
        "ownerUUID": "NNRG-04022171-cf37-b07b-b933-a6b8d57883ed",
        "ownerName": "NANoREG",
        "i5uuid": "NNRG-9204c38f-ac08-e003-bd75-97239871d602",
        "name": "Au 13 nm",
        "publicname": "Au@PBPK",
        "format": "TNOEXP",
        "substanceType": "NPO_401",
        "referenceSubstance": {
            "i5uuid": null,
            "uri": "https://apps.ideaconsult.net/nanoreg1/query/compound/search/all?search=null"
        },
        "composition": [],
        "externalIdentifiers": [
            {
                "type": "Material code",
                "id": "Au@PBPK"
            },
            {
                "type": "NANoREG supplier",
                "id": "KI"
            }
        ],
        "bundles": {},
        "studysummary": []
    }
]


#### Retrieve physchem data for selected substances

In [8]:
endpointcategory='PC_GRANULOMETRY_SECTION'
a = annotation.DictionaryEndpointCategory()
print(">>> Looking for {}".format(a.annotate(endpointcategory)))

for substance in substances:
    print(substance['URI'])    
    cli = client_ambit.AMBITSubstanceStudy(substance['URI'])
    r = cli.get(params={'category': endpointcategory,'top' : 'P-CHEM'},page=0,pagesize=10,auth=auth_object)
    #print(r.json())
    print(json.dumps(r.json(), indent=4))    

>>> Looking for http://purl.obolibrary.org/obo/CHMO_0002119
https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602
{
    "study": [
        {
            "uuid": "NRSZ-55e3f22e-f795-ae96-2ae5-a7c103503358",
            "investigation_uuid": null,
            "assay_uuid": "609caf44-b732-b054-dda5-860569e7e92e",
            "owner": {
                "substance": {
                    "uuid": "NNRG-9204c38f-ac08-e003-bd75-97239871d602"
                },
                "company": {
                    "uuid": "NNRG-04022171-cf37-b07b-b933-a6b8d57883ed",
                    "name": "NANoREG"
                }
            },
            "citation": {
                "title": "Provided",
                "year": "0",
                "owner": "KI"
            },
            "updated": "2017-08-01 00:00:00",
            "protocol": {
                "topcategory": "P-CHEM",
                "category": {
                    "code": "PC_GRANULOMETRY_SECTION",

#### Substance compositions

In [9]:
reload(client_ambit)
for substance in substances:
  
    print(substance['URI'])    
    cli = client_ambit.AMBITSubstanceComposition(substance['URI'])
    r = cli.get(auth=auth_object)
    compositions = cli.parse(r.json())
    for composition in compositions:
        print("-------------------------------------------------------------------------")
        print(composition['relation'])
        print(composition['proportion'])        
        print(composition['component']['compound']['cas'])
        print(composition['component']['compound']['name'])
        
        cli_cmp = client_ambit.AMBITCompound(root_uri=composition['component']['compound']['URI'],resource=None)
        response = cli_cmp.get(media="chemical/x-mdl-sdfile",pagesize=1)
        
        if response.status_code == 200:
            print(response.text) 
            

https://apps.ideaconsult.net/nanoreg1/substance/NNRG-9204c38f-ac08-e003-bd75-97239871d602
-------------------------------------------------------------------------
HAS_CORE
{'typical': {'precision': None, 'value': 0.0, 'unit': None}, 'real': {'lowerPrecision': None, 'lowerValue': 0.0, 'upperPrecision': None, 'upperValue': 0.0, 'unit': None}, 'function_as_additive': None}



  CDK     1129191702

  1  0  0  0  0  0  0  0  0  0999 V2000
    0.0000    0.0000    0.0000 Au  0  0  0  0  0  0  0  0  0  0  0  0
M  END
$$$$


#### Investigation
results in a tabular form

In [10]:
reload(client_ambit)
cli_investigation= client_ambit.AMBITInvestigation(service_uri)
r = cli_investigation.get(params={'search': endpointcategory,'type' : 'bystudytype'},page=0,pagesize=100,auth=auth_object)
if r.status_code==200:

    results = cli_investigation.parse(r.json())    
    print(json.dumps(results, indent=4))    
else:
    df=None
    print(r.status_code)

[
    {
        "name": "NM-402 (MWCNT 12.7 nm)",
        "publicname": "JRCNM04002a",
        "owner_name": "NANoREG",
        "topcategory": "P-CHEM",
        "endpointcategory": "PC_GRANULOMETRY_SECTION",
        "endpoint": "SIZE",
        "document_uuid": "NRDM-00000000-0000-0000-0000-000000000001",
        "guidance": "DLS",
        "reference": "final test live version",
        "reference_owner": "TNO",
        "idresult": 1807,
        "effectendpoint": "HYDRODYNAMIC DIAMETER",
        "unit": "nm",
        "loValue": 546.33,
        "errQualifier": "SD",
        "err": 0.13,
        "type_s": "study",
        "s_uuid": "NNRG-ea97c99b-e936-7dcf-b048-1ef314545e86",
        "substanceType": "NPO_354",
        "reference_year": 2016,
        "content": "JRC - IHCP",
        "iuuid": "3ECC8EA6D922C3E2B64B92F88CE81E27",
        "auuid": "D2E2DC2E0AE0FE3D12CDCDEBA31209AE",
        "updated": "2017-08-01 12:00:00",
        "studyResultType": "Measured",
        "resulttype": "Z-AVERA

In [12]:
df=pd.DataFrame(results)
display(df.head())

,name,publicname,owner_name,topcategory,endpointcategory,endpoint,document_uuid,guidance,reference,reference_owner,...,substanceType,reference_year,content,iuuid,auuid,updated,studyResultType,resulttype,resultgroup,_childDocuments_
0,NM-402 (MWCNT 12.7 nm),JRCNM04002a,NANoREG,P-CHEM,PC_GRANULOMETRY_SECTION,SIZE,NRDM-00000000-0000-0000-0000-000000000001,DLS,final test live version,TNO,...,NPO_354,2016,JRC - IHCP,3ECC8EA6D922C3E2B64B92F88CE81E27,D2E2DC2E0AE0FE3D12CDCDEBA31209AE,2017-08-01 12:00:00,Measured,Z-AVERAGE,1,[{'document_uuid': 'NRDM-00000000-0000-0000-00...
1,NM-402 (MWCNT 12.7 nm),JRCNM04002a,NANoREG,P-CHEM,PC_GRANULOMETRY_SECTION,SIZE,NRDM-00000000-0000-0000-0000-000000000001,DLS,final test live version,TNO,...,NPO_354,2016,JRC - IHCP,3ECC8EA6D922C3E2B64B92F88CE81E27,D2E2DC2E0AE0FE3D12CDCDEBA31209AE,2017-08-01 12:00:00,Measured,INTENSITY-WEIGHTED,1,[{'document_uuid': 'NRDM-00000000-0000-0000-00...
2,NM-402 (MWCNT 12.7 nm),JRCNM04002a,NANoREG,P-CHEM,PC_GRANULOMETRY_SECTION,SIZE,NRDM-00000000-0000-0000-0000-000000000001,DLS,final test live version,TNO,...,NPO_354,2016,JRC - IHCP,3ECC8EA6D922C3E2B64B92F88CE81E27,D2E2DC2E0AE0FE3D12CDCDEBA31209AE,2017-08-01 12:00:00,Measured,VOLUME-WEIGHTED,2,[{'document_uuid': 'NRDM-00000000-0000-0000-00...
3,NM-402 (MWCNT 12.7 nm),JRCNM04002a,NANoREG,P-CHEM,PC_GRANULOMETRY_SECTION,SIZE,NRDM-00000000-0000-0000-0000-000000000001,DLS,final test live version,TNO,...,NPO_354,2016,JRC - IHCP,3ECC8EA6D922C3E2B64B92F88CE81E27,D2E2DC2E0AE0FE3D12CDCDEBA31209AE,2017-08-01 12:00:00,Measured,,3,[{'document_uuid': 'NRDM-00000000-0000-0000-00...
4,NM-402 (MWCNT 12.7 nm),JRCNM04002a,NANoREG,P-CHEM,PC_GRANULOMETRY_SECTION,SIZE,NRDM-00000000-0000-0000-0000-000000000001,DLS,final test live version,TNO,...,NPO_354,2016,JRC - IHCP,3ECC8EA6D922C3E2B64B92F88CE81E27,D2E2DC2E0AE0FE3D12CDCDEBA31209AE,2017-08-01 12:00:00,Measured,INTENSITY-WEIGHTED,1,[{'document_uuid': 'NRDM-00000000-0000-0000-00...


# Aggregated search

- Using Solr-powered free text and faceted search over several eNanoMapper database instances
- see https://search.data.enanomapper.net (web app) and  https://api.ideaconsult.net for API access


### Service selection

In [13]:
print('Select enanoMapper aggregated search service:')
style = {'description_width': 'initial'}
config,config_servers, config_security, auth_object, msg = aa.parseOpenAPI3()    
service_widget = widgets.Dropdown(
    options=config_servers['url'],
    description='Service:',
    disabled=False,
    style=style
)
if config_security is None:
    service = interactive(search_service_open,url=service_widget)
else:
    print(msg)
    apikey_widget=widgets.Text(
            placeholder='',
            description=config_security,
            disabled=False,
            style=style
    )    
    service = interactive(search_service_protected,url=service_widget,apikey=apikey_widget)    

display(service)

Select enanoMapper aggregated search service:
Enter `X-Gravitee-Api-Key` you have received upon subscription to http://api.ideaconsult.net


interactive(children=(Dropdown(description='Service:', options=('https://api.ideaconsult.net/enanomapper', 'ht…

In [14]:
service_uri=service_widget.value
print("Sending queries to {}".format(service_uri))
if auth_object!=None:
    auth_object.setKey(apikey_widget.value)


Sending queries to https://api.ideaconsult.net/enanomapper


### Faceted search 

#### [Facets] Number of substances per project

In [15]:
facets = client_solr.Facets()
query=facets.getQuery(query="*:*",facets=["dbtag_hss"],fq="type_s:substance")
#print(query)
r = client_solr.post(service_uri,query=query,auth=auth_object)
response_json=r.json()
print(response_json)
if r.status_code==200:
    facets.parse(response_json['facets'])
else:
    print(r.status_code)

{'responseHeader': {'zkConnected': True, 'status': 0, 'QTime': 1, 'params': {'q': '*:*', 'json.facet': '{field1: {type:terms,field:dbtag_hss ,limit : -1, mincount:1, missing:true }}', 'fq': 'type_s:substance', 'rows': '0', 'wt': 'json'}}, 'response': {'numFound': 636, 'start': 0, 'docs': []}, 'facets': {'count': 636, 'field1': {'missing': {'count': 0}, 'buckets': [{'val': 'ENM', 'count': 636}]}}}
	()'_'	636	ALL
		('_',)'ENM'	636	field1


#### [Facets] Number of material types per project

In [16]:
query=facets.getQuery(query="*:*",facets=["dbtag_hss","substanceType_hs"],fq="type_s:substance")
#print(query)
r = client_solr.post(service_uri,query=query,auth=auth_object)
response_json=r.json()
if r.status_code==200:
    facets.parse(response_json['facets'])
else:
    print(r.status_code)

	()'_'	636	ALL
		('_',)'ENM'	636	field2
			('_', 'ENM')'NPO_401'	128	field1
			('_', 'ENM')'NPO_606'	81	field1
			('_', 'ENM')'NPO_1486'	61	field1
			('_', 'ENM')'NPO_1373'	55	field1
			('_', 'ENM')'NPO_1548'	42	field1
			('_', 'ENM')'NPO_1542'	34	field1
			('_', 'ENM')'NPO_1892'	34	field1
			('_', 'ENM')'ENM_9000006'	21	field1
			('_', 'ENM')'NPO_1550'	19	field1
			('_', 'ENM')'NPO_CdSe'	15	field1
			('_', 'ENM')'NPO_354'	12	field1
			('_', 'ENM')'ENM_9000005'	10	field1
			('_', 'ENM')'NPO_1544'	10	field1
			('_', 'ENM')'NPO_199'	9	field1
			('_', 'ENM')'CHEBI_33416'	8	field1
			('_', 'ENM')'NPO_1384'	6	field1
			('_', 'ENM')'NPO_HfO2'	6	field1
			('_', 'ENM')'NPO_602'	5	field1
			('_', 'ENM')'NPO_Co3O4'	5	field1
			('_', 'ENM')'NPO_NiO'	5	field1
			('_', 'ENM')'NPO_1862'	4	field1
			('_', 'ENM')'NPO_Cr2O3'	4	field1
			('_', 'ENM')'NPO_SnO2'	4	field1
			('_', 'ENM')'NPO_ZrO2'	4	field1
			('_', 'ENM')'CHEBI_36973'	3	field1
			('_', 'ENM')'NPO_In2O3'	3	field1
			('_', 'ENM')'NPO_La2O3'	

In [17]:
a = annotation.DictionarySubstancetypes()
term=a.annotate("NPO_354")
print(term)
term=a.annotate("NPO_1373")
print(term)


multi-walled carbon nanotube
silicon dioxide nanoparticle


#### [Facets] Get all cell types

In [18]:
reload(client_solr)

facets = client_solr.Facets()
query=facets.getQuery(query="*:*",facets=["E.cell_type_s"],fq="type_s:params")
#print(query)
r = client_solr.post(service_uri,query=query,auth=auth_object)
response_json=r.json()
if r.status_code==200:
    facets.parse(response_json['facets'])
else:
    print(r.status_code)


	()'_'	4488	ALL
		('_',)'_'	4488	field1


#### [Facets] Get all protocols per endpoint for titanium dioxide nanoparticles (NPO_1486)

In [19]:
fields=["topcategory_s","endpointcategory_s","guidance_s"]
query=facets.getQuery(query="substanceType_s:NPO_1486",fq="type_s:study",facets=fields)
print(query)
r = client_solr.post(service_uri,query=query,auth=auth_object)
print(r.status_code)
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

{'q': 'substanceType_s:NPO_1486', 'fq': 'type_s:study', 'wt': 'json', 'json.facet': '{field3: {type:terms,field:topcategory_s ,limit : -1, mincount:1, missing:true , facet:{field2: {type:terms,field:endpointcategory_s ,limit : -1, mincount:1, missing:true , facet:{field1: {type:terms,field:guidance_s ,limit : -1, mincount:1, missing:true }}}}}}', 'rows': 0}
200
	()'_'	480	ALL
		('_',)'TOX'	273	field3
			('_', 'TOX')'NPO_1709_SECTION'	114	field2
				('_', 'TOX', 'NPO_1709_SECTION')'CYTOTOXICITY USING HUMAN MONOCYTE-DERIVED MACROPHAGES (HMDM)'	96	field1
				('_', 'TOX', 'NPO_1709_SECTION')'LDH'	18	field1
			('_', 'TOX')'BAO_0002993_SECTION'	64	field2
				('_', 'TOX', 'BAO_0002993_SECTION')'ELISA / TNF-Α RELEASE IN CULTURE MEDIUM'	64	field1
			('_', 'TOX')'BAO_0003009_SECTION'	60	field2
				('_', 'TOX', 'BAO_0003009_SECTION')'WST-1'	45	field1
				('_', 'TOX', 'BAO_0003009_SECTION')'PI UPTAKE ASSAY'	9	field1
				('_', 'TOX', 'BAO_0003009_SECTION')'LACTATE DEHYDROGENASE ACTIVITY'	2	field1
	

#### [Facets] Get all methods

In [20]:
fields=["topcategory_s","endpointcategory_s","E.method_s","E.sop_reference_s"]
query=facets.getQuery(query="*:*",fq="type_s:params",facets=fields)
print(query)
r = client_solr.post(service_uri,query=query,auth=auth_object)
print(r.status_code)
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

{'q': '*:*', 'fq': 'type_s:params', 'wt': 'json', 'json.facet': '{field4: {type:terms,field:topcategory_s ,limit : -1, mincount:1, missing:true , facet:{field3: {type:terms,field:endpointcategory_s ,limit : -1, mincount:1, missing:true , facet:{field2: {type:terms,field:E.method_s ,limit : -1, mincount:1, missing:true , facet:{field1: {type:terms,field:E.sop_reference_s ,limit : -1, mincount:1, missing:true }}}}}}}}', 'rows': 0}
200
	()'_'	4488	ALL
		('_',)'P-CHEM'	2775	field4
			('_', 'P-CHEM')'PC_GRANULOMETRY_SECTION'	1611	field3
				('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION')'_'	1611	field2
					('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION', '_')'_'	1611	field1
			('_', 'P-CHEM')'PC_UNKNOWN_SECTION'	679	field3
				('_', 'P-CHEM', 'PC_UNKNOWN_SECTION')'_'	679	field2
					('_', 'P-CHEM', 'PC_UNKNOWN_SECTION', '_')'_'	679	field1
			('_', 'P-CHEM')'ZETA_POTENTIAL_SECTION'	214	field3
				('_', 'P-CHEM', 'ZETA_POTENTIAL_SECTION')'_'	214	field2
					('_', 'P-CHEM', 'ZETA_POTENTIAL_SECTION', '_'

#### [Facets] Get all material types

In [21]:
fields=["substanceType_hs","publicname_hs","name_hs","dbtag_hss"]
query=facets.getQuery(fq="type_s:substance",facets=fields)
#print(query)
r = client_solr.post(service_uri,query=query,auth=auth_object)
print(r.status_code)
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

200
	()'_'	636	ALL
		('_',)'NPO_401'	128	field4
			('_', 'NPO_401')'Au'	2	field3
				('_', 'NPO_401', 'Au')'Gioria2016 30'	1	field2
					('_', 'NPO_401', 'Au', 'Gioria2016 30')'ENM'	1	field1
				('_', 'NPO_401', 'Au')'Gioria2016 5'	1	field2
					('_', 'NPO_401', 'Au', 'Gioria2016 5')'ENM'	1	field1
			('_', 'NPO_401')'Ag NM-300'	1	field3
				('_', 'NPO_401', 'Ag NM-300')'Ag NM-300_95nm_0_Vortexing'	1	field2
					('_', 'NPO_401', 'Ag NM-300', 'Ag NM-300_95nm_0_Vortexing')'ENM'	1	field1
			('_', 'NPO_401')'Ag NM-300k'	1	field3
				('_', 'NPO_401', 'Ag NM-300k')'Ag NM-300k_65nm_0.1_Bath'	1	field2
					('_', 'NPO_401', 'Ag NM-300k', 'Ag NM-300k_65nm_0.1_Bath')'ENM'	1	field1
			('_', 'NPO_401')'Ag20'	1	field3
				('_', 'NPO_401', 'Ag20')'Ag20_105nm_0.1_Vortexing'	1	field2
					('_', 'NPO_401', 'Ag20', 'Ag20_105nm_0.1_Vortexing')'ENM'	1	field1
			('_', 'NPO_401')'Ag20Cit'	1	field3
				('_', 'NPO_401', 'Ag20Cit')'Ag20Cit_18nm_0.1_Bath'	1	field2
					('_', 'NPO_401', 'Ag20Cit', 'Ag20Cit_18nm_0.

#### [Facets]  Get all endpoints for nanotubes

In [22]:
query=facets.getQuery(query="carbon nanotube",facets=["topcategory_s","endpointcategory_s","effectendpoint_s","unit_s"],fq="type_s:study")
#print(query)
r = client_solr.post(service_uri,query=query,auth=auth_object)
print(r.status_code)
#print(r.json()['facets'])
if r.status_code==200:
    facets.parse(r.json()['facets'])
else:
    print(r.status_code)

200
	()'_'	52	ALL
		('_',)'P-CHEM'	41	field4
			('_', 'P-CHEM')'ZETA_POTENTIAL_SECTION'	28	field3
				('_', 'P-CHEM', 'ZETA_POTENTIAL_SECTION')'ZETA POTENTIAL'	28	field2
					('_', 'P-CHEM', 'ZETA_POTENTIAL_SECTION', 'ZETA POTENTIAL')'mV'	28	field1
			('_', 'P-CHEM')'PC_GRANULOMETRY_SECTION'	7	field3
				('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION')'AVERAGE LENGTH'	3	field2
					('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION', 'AVERAGE LENGTH')'nm'	3	field1
				('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION')'INNER_DIAMETER'	1	field2
					('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION', 'INNER_DIAMETER')'nm'	1	field1
				('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION')'MASS MEDIAN AERODYNAMIC DIAMETER'	1	field2
					('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION', 'MASS MEDIAN AERODYNAMIC DIAMETER')''	1	field1
				('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION')'MASS MEDIAN DIAMETER'	1	field2
					('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION', 'MASS MEDIAN DIAMETER')'nm'	1	field1
				('_', 'P-CHEM', 'PC_GRANULOMETRY_SECTION')'P

### Retrieve experimental data

#### Physchem example - MWCNT size

In [23]:
reload(client_solr)
study = client_solr.StudyDocuments()
filter = {'topcategory_s':'P-CHEM', 'endpointcategory_s':'PC_GRANULOMETRY_SECTION' }
study.setStudyFilter(filter)
print(study.getSettings())
#all TiO2 NPO_1486
query = study.getQuery(textfilter='substanceType_s:NPO_354',rows=10000)
r = client_solr.post(service_uri,query=query,auth=auth_object)

{'studyfilter': '   topcategory_s:P-CHEM AND endpointcategory_s:PC_GRANULOMETRY_SECTION', 'query_organism': None, 'endpointfilter': None, 'query_guidance': None, 'fields': None}


In [25]:
#parse the data
if r.status_code==200:
    study = client_solr.StudyDocuments()
    rows = study.parse(r.json()['response']['docs'])
    df = study.rows2frame(rows)
    rows=None
    uuids = ['uuid.investigation','uuid.assay','uuid.document','uuid.substance']
    df.sort_values(by=uuids)
    display(df.head(50))
else:
    print(r.status_code)

,db,m.substance.name,m.public.name,m.materialprovider,m.substance.type,p.oht.module,p.oht.section,p.guidance,p.reference,p.reference_year,...,xR.reliability,xR.studyResultType,xR.purposeFlag,xx.QualityRemark,uuid.substance,uuid.document,uuid.assay,uuid.investigation,x.params.Method type,x.params.guidance
0,ENM,Nymark2015-M1,MWCNT Mitsui-7,NanoWiki,NPO_354,P-CHEM,PC_GRANULOMETRY_SECTION,,http://dx.doi.org/10.3109/17435390.2015.1017022,2015,...,NaN,,,,NWKI-05ce8604-4efc-376b-a88f-338a90465243,NWKI-87da6899-6d40-419e-a22a-83ca05cf3bad,NWKI-87da6899-6d40-419e-a22a-83ca05cf3bad,87da6899-6d40-419e-a22a-83ca05cf3bad,NaN,NaN
1,ENM,Tilton2014 M1,MWCNT,NanoWiki,NPO_354,P-CHEM,PC_GRANULOMETRY_SECTION,TEM,http://dx.doi.org/10.3109/17435390.2013.803624,2014,...,NaN,,,,NWKI-b28c0a45-e09d-35c6-b78c-359e174aad58,NWKI-095b9c55-61e5-45c9-8ce1-24e4eff4f0e7,NWKI-095b9c55-61e5-45c9-8ce1-24e4eff4f0e7,095b9c55-61e5-45c9-8ce1-24e4eff4f0e7,TEM,TEM


In [29]:
#Group by material and endpoint
groups=[]

groups.append("m.public.name")
#groups.append("x.params.E.method")
#groups.append("p.guidance")
groups.append("x.params.MEDIUM")
groups.append("value.endpoint")
groups.append("value.endpoint_type")
groups.append("value.unit")
print(groups)

tmp=df.groupby(by=groups).agg({"value.range.lo" : ["mean","std","count"]}).reset_index()
(tmp)

['m.public.name', 'x.params.MEDIUM', 'value.endpoint', 'value.endpoint_type', 'value.unit']


KeyError: 'm.public.name'

#### Tox example - TiO2 cell viability

In [30]:
reload(client_solr)
study = client_solr.StudyDocuments()
filter = {'topcategory_s':'TOX', 'endpointcategory_s':'ENM_0000068_SECTION' }
study.setStudyFilter(filter)
print(study.getSettings())
#all TiO2 NPO_1486
query = study.getQuery(textfilter='substanceType_s:NPO_354',rows=10000)
r = client_solr.post(service_uri,query=query,auth=auth_object)

{'studyfilter': '   topcategory_s:TOX AND endpointcategory_s:ENM_0000068_SECTION', 'query_organism': None, 'endpointfilter': None, 'query_guidance': None, 'fields': None}


In [31]:
#parse the data
if r.status_code==200:
    study = client_solr.StudyDocuments()
    rows = study.parse(r.json()['response']['docs'])
    df = study.rows2frame(rows)
    rows=None
    uuids = ['uuid.investigation','uuid.assay','uuid.document','uuid.substance']
    df.sort_values(by=uuids)
    display(df.head(50))
else:
    print(r.status_code)

KeyError: 'uuid.investigation'

In [32]:
groups=[]

groups.append("m.public.name")
groups.append("uuid.assay")
groups.append("uuid.document")
#groups.append("x.params.E.method")
#groups.append("p.guidance")
groups.append("x.params.MEDIUM")
groups.append("x.params.E.cell_type")
groups.append("x.conditions.material")
groups.append("value.endpoint")
groups.append("value.endpoint_type")
groups.append("value.unit")
print(groups)

tmp=df.groupby(by=groups).agg({"value.range.lo" : ["mean","std","count"]}).reset_index()
display(tmp)

['m.public.name', 'uuid.assay', 'uuid.document', 'x.params.MEDIUM', 'x.params.E.cell_type', 'x.conditions.material', 'value.endpoint', 'value.endpoint_type', 'value.unit']


KeyError: 'm.public.name'

.

## Annotation examples

In [33]:

reload(annotation)
a = annotation.DictionaryEndpoints()
for endpoint in ["CIRCULARITY","FERET_DIAMETER","IC50"]:
    term=a.annotate(endpoint)
    print(endpoint)
    print(term)
    print(a.getLink(term))


CIRCULARITY
http://purl.enanomapper.org/onto/ENM_8000038
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.enanomapper.org/onto/ENM_8000038
FERET_DIAMETER
http://purl.enanomapper.org/onto/ENM_8000040
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.enanomapper.org/onto/ENM_8000040
IC50
http://www.bioassayontology.org/bao#BAO_0000190
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://www.bioassayontology.org/bao#BAO_0000190


In [34]:
a = annotation.DictionaryCells()
for cell in ["3T3","A549"]:
    term=a.annotate(cell)
    print(cell)
    print(term)
    print(a.getLink(term))

3T3
http://purl.obolibrary.org/obo/CLO_0001345
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.obolibrary.org/obo/CLO_0001345
A549
http://purl.obolibrary.org/obo/CLO_0001601
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.obolibrary.org/obo/CLO_0001601


In [35]:
a = annotation.DictionaryAssays()
for assay in ["CFE","Alamar blue","TEM","COMET"]:
    term=a.annotate(assay)
    print(assay)
    print(term)
    print(a.getLink(term))


CFE
http://purl.enanomapper.org/onto/ENM_8000235
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.enanomapper.org/onto/ENM_8000235
Alamar blue
Alamar blue
None
TEM
http://purl.bioontology.org/ontology/npo/NPO_1430
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.bioontology.org/ontology/npo/NPO_1430
COMET
http://purl.obolibrary.org/obo/OBI_0302736
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.obolibrary.org/obo/OBI_0302736


In [36]:
a = annotation.DictionaryEndpointCategory()
term=a.annotate("PC_GRANULOMETRY_SECTION")
print(term)
print(a.getLink(term))

http://purl.obolibrary.org/obo/CHMO_0002119
http://bioportal.bioontology.org/ontologies/ENM/?p=classes&conceptid=http://purl.obolibrary.org/obo/CHMO_0002119


In [37]:
a = annotation.DictionarySpecies()
term=a.annotate("rat")
print(term)
print(a.getLink(term))

rat
None


In [38]:
a = annotation.DictionarySubstancetypes()
term=a.annotate("NPO_401")
print(term)


gold nanoparticle
